In [ ]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.sparse as SC
import scipy.sparse.linalg as linalg
from PixelLocation import ROI_points
from PixelLocation import pixel_location
from PixelLocation import surrounding_pixels
from PixelLocation import in_omega
from sparse_matrix import sparse_matrix
from Laplacian_of_ROI import laplace
from Selecting_ROI import mousepoints
from Selecting_ROI import get_omega
from create_mask import create_mask


In [ ]:
points=[] # holds the values of the selected coordinates
def mousepoints(events,x, y,flags,parameters):
    global points
    
    if events==cv2.EVENT_LBUTTONDOWN: 
        points.append((x,y))

    return

def get_omega(im):
    global points

    cv2.imshow("Please Click on image to get ROI points",im)
    cv2.setMouseCallback("Please Click on image to get ROI points", mousepoints)
    cv2.waitKey(0)


    photo = im.copy() # making a copy of our image for visulalisation
    new_img=cv2.polylines(photo, np.int32([points]), isClosed=True, color=(0,0,255),thickness=4) # draw a polygon of our region of interest

    return points, photo, new_img

In [ ]:
#Click on the image to generate the mask

source_path=r"inputs\1.filling\hope.jpg" #https://www.joom.com/en/products/5bc6eb7c28fc710198a726a1
target_path=r"inputs\1.filling\Me.jpg" #https://unsplash.com/images/nature/space

src= cv2.imread(source_path,cv2.IMREAD_GRAYSCALE)
target=cv2.imread(target_path,cv2.IMREAD_GRAYSCALE)

# resizing the images because the larger the images the more time it takes for the A matrix to computed
src= cv2.resize(src,(200,150))
target=cv2.resize(target,(200,150))

points.clear()#clearing the list to put new values

boundary=get_omega(target)
mask=create_mask(target,boundary[0])

cv2.imshow("src mask",mask[0])# show created mask
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite(r"inputs\1.filling\fill mask.jpg", mask[0]) # Save mask into same path as src and target


In [ ]:
def fill(src,target,mask):

    Indicies=ROI_points(mask)
    A_Mat=sparse_matrix(Indicies)
    M= len(Indicies)
    
    B_Mat= np.zeros(M) # This is the B matrix
    
    for i, index in enumerate(Indicies):
        
        B_Mat[i]=laplace(src,index)

        if pixel_location(index,mask): #checking if pixel is on the boundary
            for pixels in surrounding_pixels(index): #the surrounding pixels
                if not np.any(in_omega(pixels,mask)): # pixels npt in ROI
                    B_Mat[i] += target[pixels]

    x=linalg.cg(A_Mat,B_Mat) #solve for x to get new pixels
    new_image=np.copy(target).astype(int)
    for i, index in enumerate(Indicies):
        new_image[index]=x[0][i]

    return new_image


mask=cv2.imread(r"inputs\1.filling\fill mask.jpg")
if __name__ == '__main__':
    mask=mask.astype(np.float64)/255
    mask[mask !=1] =0

    new_image=[fill(src,target,mask)]
    new=cv2.merge(new_image)
    new = new.astype(np.uint8)

    new= cv2.imwrite(r"Results\1.filling\output.jpg", new)

